In [4]:
import pandas as pd
import json
import os
import re

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO
from tqdm import tqdm
import os

In [7]:
def load_data(file, sheet_name=None, encoding='utf-8'):
    """
    Load data from a CSV, Excel, or JSON file.

    Parameters:
    - file: str, path to the file
    - sheet_name: str, sheet name if the file is an Excel file (default is None)
    - encoding: str, encoding of the file (default is 'utf-8')

    Returns:
    - df: pandas DataFrame
    """
    if file.endswith('.csv'):
        df = pd.read_csv(file, encoding=encoding)
    elif file.endswith('.xlsx'):
        df = pd.read_excel(file, sheet_name=sheet_name, encoding=encoding)
    elif file.endswith('.json'):
        df = pd.read_json(file, encoding=encoding)
    else:
        raise ValueError("Unsupported file format")
    return df

def statistics(df):
    """
    Generate basic statistics for the DataFrame.

    Parameters:
    - df: pandas DataFrame

    Returns:
    - desc: DataFrame, descriptive statistics
    - info: None, prints info about the DataFrame
    - unique_counts: Series, count of unique values per column
    """
    desc = df.describe()
    info = df.info()
    unique_counts = df.nunique()
    return desc, info, unique_counts

def missing_values(df, save_to_memory=True):
    """
    Perform missing values analysis and plot the results.

    Parameters:
    - df: pandas DataFrame
    - save_to_memory: bool, whether to save the plot in memory or locally (default is True)

    Returns:
    - missing_df: DataFrame, missing values and their percentages
    - buf or filepath: BytesIO buffer or str, path to the saved plot
    """
    missing = df.isnull().sum()
    total = df.shape[0]
    missing_percentage = (missing / total) * 100
    missing_df = pd.DataFrame({'Missing Values': missing, 'Percentage': missing_percentage})
    
    # Plotting
    fig, ax = plt.subplots(figsize=(10, 8))
    missing_df.plot(kind='barh', stacked=True, ax=ax)
    plt.title('Missing Values Analysis')
    plt.xlabel('Count')
    if save_to_memory:
        buf = BytesIO()
        plt.savefig(buf, format='png')
        plt.close()
        buf.seek(0)
        return missing_df, buf
    else:
        plt.savefig('missing_values.png')
        plt.close()
        return missing_df, 'missing_values.png'

def univariate_analysis(df, nunique_threshold=15, save_to_memory=True):
    """
    Perform univariate analysis and plot the results.

    Parameters:
    - df: pandas DataFrame
    - nunique_threshold: int, threshold for number of unique values to consider for countplot (default is 15)
    - save_to_memory: bool, whether to save the plots in memory or locally (default is True)

    Returns:
    - plots: list of tuples, containing plot titles and BytesIO buffers or file paths
    """
    plots = []
    for column in tqdm(df.columns, desc="Univariate Analysis"):
        if df[column].dtype == 'object' and df[column].nunique() < nunique_threshold:
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.countplot(y=df[column], ax=ax)
            plt.title(f'Count Plot of {column}')
            if save_to_memory:
                buf = BytesIO()
                plt.savefig(buf, format='png')
                plt.close()
                buf.seek(0)
                plots.append((f'Count Plot of {column}', buf))
            else:
                if not os.path.exists('univariate'):
                    os.makedirs('univariate')
                plt.savefig(f'univariate/countplot_{column}.png')
                plt.close()
                plots.append((f'Count Plot of {column}', f'univariate/countplot_{column}.png'))
        elif df[column].dtype != 'object':
            fig, axs = plt.subplots(1, 2, figsize=(14, 6))
            sns.histplot(df[column], kde=True, ax=axs[0])
            axs[0].set_title(f'Histogram of {column}')
            sns.boxplot(x=df[column], ax=axs[1])
            axs[1].set_title(f'Boxplot of {column}')
            if save_to_memory:
                buf = BytesIO()
                plt.savefig(buf, format='png')
                plt.close()
                buf.seek(0)
                plots.append((f'Univariate Analysis of {column}', buf))
            else:
                if not os.path.exists('univariate'):
                    os.makedirs('univariate')
                plt.savefig(f'univariate/univariate_{column}.png')
                plt.close()
                plots.append((f'Univariate Analysis of {column}', f'univariate/univariate_{column}.png'))
    return plots

def bivariate_analysis(df, nunique_threshold=15, save_to_memory=True):
    """
    Perform bivariate analysis and plot the results.

    Parameters:
    - df: pandas DataFrame
    - nunique_threshold: int, threshold for number of unique values to consider for categorical plots (default is 15)
    - save_to_memory: bool, whether to save the plots in memory or locally (default is True)

    Returns:
    - plots: list of tuples, containing plot titles and BytesIO buffers or file paths
    """
    plots = []
    for column in tqdm(df.columns, desc="Bivariate Analysis"):
        if df[column].dtype == 'object' and df[column].nunique() < nunique_threshold:
            for num_column in df.select_dtypes(include=np.number).columns:
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.boxplot(x=df[column], y=df[num_column], ax=ax)
                plt.title(f'{column} vs {num_column}')
                if save_to_memory:
                    buf = BytesIO()
                    plt.savefig(buf, format='png')
                    plt.close()
                    buf.seek(0)
                    plots.append((f'{column} vs {num_column}', buf))
                else:
                    if not os.path.exists('bivariate'):
                        os.makedirs('bivariate')
                    plt.savefig(f'bivariate/bivariate_{column}_vs_{num_column}.png')
                    plt.close()
                    plots.append((f'{column} vs {num_column}', f'bivariate/bivariate_{column}_vs_{num_column}.png'))
    return plots

def plot_corr(df, annot=True, cmap='RdYlGn', center=0, square=True, save_to_memory=True):
    """
    Plot a heatmap correlation plot.

    Parameters:
    - df: pandas DataFrame
    - annot: bool, whether to annotate the heatmap with correlation values (default is True)
    - cmap: str, colormap for the heatmap (default is 'RdYlGn')
    - center: float, center value for the colormap (default is 0)
    - square: bool, whether to force the plot to be square (default is True)
    - save_to_memory: bool, whether to save the plot in memory or locally (default is True)

    Returns:
    - buf or filepath: BytesIO buffer or str, path to the saved plot
    """
    num_df = df.select_dtypes(include=np.number)
    corr = num_df.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(corr, mask=mask, cmap=cmap, annot=annot, center=center, square=square)
    plt.tight_layout()
    if save_to_memory:
        buf = BytesIO()
        plt.savefig(buf, format='png')
        plt.close()
        buf.seek(0)
        return buf
    else:
        if not os.path.exists('multivariate'):
            os.makedirs('multivariate')
        plt.savefig('multivariate/correlation_heatmap.png')
        plt.close()
        return 'multivariate/correlation_heatmap.png'

def create_pptx(missing_buf, univariate_plots, bivariate_plots, corr_buf):
    """
    Create a PowerPoint presentation with the generated plots.

    Parameters:
    - missing_buf: BytesIO buffer, plot buffer for missing values
    - univariate_plots: list of tuples, plot buffers for univariate analysis
    - bivariate_plots: list of tuples, plot buffers for bivariate analysis
    - corr_buf: BytesIO buffer, plot buffer for correlation heatmap

    Returns:
    - None, saves the PowerPoint presentation as 'EDA_Report.pptx'
    """
    prs = Presentation()
    
    # Slide for missing values
    slide_layout = prs.slide_layouts[5] # Use a title and content layout
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = "Missing Values Analysis"
    slide.shapes.add_picture(missing_buf, Inches(1), Inches(1.5), height=Inches(5.5))

    # Slides for univariate analysis
    for title_text, plot_buf in univariate_plots:
        slide = prs.slides.add_slide(slide_layout)
        title = slide.shapes.title
        title.text = title_text
        slide.shapes.add_picture(plot_buf, Inches(1), Inches(1.5), height=Inches(5.5))

    # Slides for bivariate analysis
    for title_text, plot_buf in bivariate_plots:
        slide = prs.slides.add_slide(slide_layout)
        title = slide.shapes.title
        title.text = title_text
        slide.shapes.add_picture(plot_buf, Inches(1), Inches(1.5), height=Inches(5.5))

    # Slide for correlation heatmap
    slide = prs.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = "Correlation Heatmap"
    slide.shapes.add_picture(corr_buf, Inches(1), Inches(1.5), height=Inches(5.5))
    
    prs.save('EDA_Report.pptx')

def main(file, sheet_name=None, encoding='utf-8', nunique_threshold=15, save_to_memory=True, perform_univariate=True, perform_bivariate=True, perform_multivariate=True, columns=None):
    """
    Main function to perform Exploratory Data Analysis (EDA) on a given dataset.

    Parameters:
    - file (str): Path to the dataset file.
    - sheet_name (str, optional): Name of the sheet in an Excel file. Default is None.
    - encoding (str, optional): Encoding of the dataset file. Default is 'utf-8'.
    - nunique_threshold (int, optional): Threshold for the number of unique values to consider for countplot. Default is 15.
    - save_to_memory (bool, optional): Whether to save the plots in memory or locally. Default is True.
    - perform_univariate (bool, optional): Whether to perform univariate analysis. Default is True.
    - perform_bivariate (bool, optional): Whether to perform bivariate analysis. Default is True.
    - perform_multivariate (bool, optional): Whether to perform multivariate analysis. Default is True.
    - columns (list, optional): List of column names to consider for analysis. Default is None.

    Returns:
    - None: Prints a success message when the EDA report is generated.
    """
    df = load_data(file, sheet_name, encoding)
    
    if columns:
        df = df[columns]
    
    desc, info, unique_counts = statistics(df)
    missing_df, missing_buf = missing_values(df, save_to_memory)
    
    univariate_plots = []
    if perform_univariate:
        univariate_plots = univariate_analysis(df, nunique_threshold, save_to_memory)
    
    bivariate_plots = []
    if perform_bivariate:
        bivariate_plots = bivariate_analysis(df, nunique_threshold, save_to_memory)
    
    corr_buf = None
    if perform_multivariate:
        corr_buf = plot_corr(df, save_to_memory=save_to_memory)
    
    create_pptx(missing_buf, univariate_plots, bivariate_plots, corr_buf)
    print("EDA Report generated successfully as 'EDA_Report.pptx'")

# Run the pipeline
file = r"C:\Users\Asif Sayyed\Documents\GitHub\MachineLearns\data\auto-mpg.csv"  # replace with your file path
main(file, nunique_threshold=15, save_to_memory=False, perform_univariate=True, perform_bivariate=True, perform_multivariate=True, columns=None)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


Bivariate Analysis: 100%|██████████| 9/9 [00:00<00:00, 5975.74it/s]


EDA Report generated successfully as 'EDA_Report.pptx'


In [10]:
def load_data(file_path, encoding='utf-8'):
    """
    Load data from a specified file path. Supports CSV, Excel, JSON, and Parquet files.
    
    Parameters:
    - file_path (str): Path to the file to be loaded.
    - encoding (str): Encoding to be used for reading the file. Default is 'utf-8'.
    
    Returns:
    - dict or DataFrame: A dictionary of DataFrames with keys being sheet names for Excel files,
                         or a single DataFrame for other file types.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file at {file_path} does not exist.")
    
    file_extension = os.path.splitext(file_path)[1].lower()
    
    try:
        if file_extension == '.csv':
            data = pd.read_csv(file_path, encoding=encoding)
            data.columns = data.columns.str.lower()
        elif file_extension in ['.xls', '.xlsx']:
            data = pd.read_excel(file_path, sheet_name=None)
        elif file_extension == '.json':
            with open(file_path, 'r', encoding=encoding) as f:
                json_data = json.load(f)
            data = pd.json_normalize(json_data)
            data.columns = data.columns.str.lower()
        elif file_extension == '.parquet':
            data = pd.read_parquet(file_path)
        else:
            raise ValueError(f"Unsupported file extension: {file_extension}")
    except UnicodeDecodeError as e:
        raise ValueError(f"Error loading the file due to encoding issues: {e}")
    except Exception as e:
        raise ValueError(f"Error loading the file: {e}")
    
    return data

def sanitize_sheet_name(sheet_name):
    """
    Sanitize the sheet name to be a valid Python variable name and convert to lowercase.
    
    Parameters:
    - sheet_name (str): Original sheet name.
    
    Returns:
    - str: Sanitized sheet name.
    """
    return re.sub(r'\W|^(?=\d)', '_', sheet_name.lower())

In [24]:
def check_df(dataframe, head=5):
    """
    Function to check basic characteristics of a DataFrame.

    Parameters:
    ----------
    - dataframe: pandas.core.DataFrame
        DataFrame to be checked.
    - head: int
        Number of rows to display from the beginning and end of the DataFrame.

    Returns:
    -------
    None

    Example Usage:
    --------------
    check_df(df, head=10)
    """
    print("################## Shape ####################")
    print(dataframe.shape)
    print("################## Types ####################")
    print(dataframe.dtypes)
    print("################## Head ####################")
    display(dataframe.head(head))
    print("################## Tail ####################")
    display(dataframe.tail(head))
    print("################## NA ####################")
    print(dataframe.isnull().sum())
    print("################## Quantiles ####################")
    display(dataframe.describe([0.05, 0.10, 0.25, 0.40, 0.50, 0.60, 0.75, 0.90, 0.95]).T)

In [25]:
# path = r"D:\College\Academics\Extra\Datasets\daily-total-female-births.csv"
# path = r"D:\College\Academics\Extra\Datasets\test.xlsx"
path = r"D:\College\Academics\SEM 4\New Generation Database\Datasets\playstore.json"

# Helper Code

In [26]:
#reading csv or json
df = load_data(path)
display(df.head())

# excel_data = load_data(path)
# for sheet_name, df in excel_data.items():
#     sanitized_name = sanitize_sheet_name(sheet_name)
#     globals()[sanitized_name] = df
#     print(f"DataFrame '{sanitized_name}' created:")
#     display(globals()[sanitized_name].head())

,date,births
0,1959-01-01,35
1,1959-01-02,32
2,1959-01-03,30
3,1959-01-04,31
4,1959-01-05,44


In [28]:
check_df(df)

################## Shape ####################
(365, 2)
################## Types ####################
date      object
births     int64
dtype: object
################## Head ####################


,date,births
0,1959-01-01,35
1,1959-01-02,32
2,1959-01-03,30
3,1959-01-04,31
4,1959-01-05,44


################## Tail ####################


,date,births
360,1959-12-27,37
361,1959-12-28,52
362,1959-12-29,48
363,1959-12-30,55
364,1959-12-31,50


################## NA ####################
date      0
births    0
dtype: int64
################## Quantiles ####################


,count,mean,std,min,5%,10%,25%,40%,50%,60%,75%,90%,95%,max
births,365.0,41.980822,7.348257,23.0,31.0,33.4,37.0,40.0,42.0,44.0,46.0,51.6,55.0,73.0


: 